In [60]:
from gensim.models import Word2Vec

In [61]:
model_word_to_vec = Word2Vec.load('word2vec.model')

In [62]:
idx2tag={0: 'O',
         1: 'B-LOC',
         2: 'B-GRP',
         3: 'I-GRP',
         4: 'B-PROD',
         5: 'B-CW',
         6: 'I-CW',
         7: 'B-CORP',
         8: 'B-PER',
         9: 'I-PER',
         10: 'I-CORP',
         11: 'I-PROD',
         12: 'I-LOC'}

In [63]:
tag2idx={'O': 0,
         'B-LOC': 1,
         'B-GRP': 2,
         'I-GRP': 3,
         'B-PROD': 4,
         'B-CW': 5,
         'I-CW': 6,
         'B-CORP': 7,
         'B-PER': 8,
         'I-PER': 9,
         'I-CORP': 10,
         'I-PROD': 11,
         'I-LOC': 12}

In [64]:
punctuations = '''“”!()-[]{};:'"\,<>./?@#$%^&*_~�।’‘1234567890''' #we can add suitable extra punctuation all the time
def remove_punctuation(d):
    review = d.replace('\n', '')
    no_punct = ""
    for char in review:
        if char not in punctuations:
            no_punct = no_punct + char
    return no_punct

In [65]:
def input_file(fileName):
    word = [[]]
    sentences = [[]]
    target = [[]]
    count_class = {}
    i=0
    with open(fileName, 'r') as file:
        for line in file:
            l = line.split()
            if len(l):
                if l[0] not in punctuations:
                    l[0] = remove_punctuation(l[0])
                    word[i].append((l[0], l[-1]))
                    target[i].append(l[-1])
                    sentences[i].append(l[0])
                    if l[-1] not in count_class.keys():
                        count_class.update({l[-1]:1})
                    else:
                        count_class[l[-1]] += 1
                else:
                    word[i].append((l[0], l[-1]))
                    sentences[i].append(l[0])
            else:
                word.append([])
                sentences.append([])
                target.append([])
                i+=1
    
    return word, sentences, target, count_class

In [66]:
train_token, train_sentence, train_target, count_class_train = input_file('train.txt')

In [67]:
dev_token, dev_sentence, dev_target, count_class_dev = input_file('dev.txt')

In [68]:
mxlen = 50

In [69]:
train_sentence.append(['unknown'])
train_target.append(['O'])

In [70]:
vocab = train_sentence + dev_sentence
flat_list = [item for sublist in vocab for item in sublist]
unique_words = list(set(x for x in flat_list))
len_unique_words = len(unique_words)
len_unique_words

29176

In [71]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

x = [[model_word_to_vec.wv.get_vector(x) for x in s] for s in train_sentence]
x = pad_sequences(maxlen = mxlen+5, sequences=x, padding='post', value=len_unique_words-1)

In [72]:
labels = list(count_class_train.keys())
num_classes = len(labels)
num_classes

13

In [73]:
y = [[tag2idx[w] for w in s] for s in train_target]
y = pad_sequences(maxlen = mxlen+5, sequences=y, padding='post', value=tag2idx["O"])
y = [to_categorical(i, num_classes=num_classes) for i in y]

In [74]:
x.shape

(15301, 55, 100)

In [75]:
len(y)

15301

In [76]:
x_input = [elem for twod in x for elem in twod]

In [77]:
len(x_input)

841555

In [78]:
y_input = [elem for twod in y for elem in twod]

In [79]:
len(y_input)

841555

In [80]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

rf = RandomForestClassifier()

In [81]:
rf.fit(x_input, y_input)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [82]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

x = [[model_word_to_vec.wv.get_vector(x) for x in s] for s in dev_sentence]
x = pad_sequences(maxlen = mxlen+5, sequences=x, padding='post', value=len_unique_words-1)

In [83]:
x = [elem for twod in x for elem in twod]

In [84]:
y_pred = rf.predict(x)

In [86]:
y_pred.shape

(44000, 13)

In [92]:
# initializing K 
K = 55
  
# Convert 2D list to 3D at K slicing
# Using loop
res = []
subl = []
cnt = 0
for sub in y_pred:
    subl.append(sub)
    cnt = cnt + 1
    if cnt >= K:
        res.append(subl)
        subl = []
        cnt = 0

In [93]:
len(res)

800

In [96]:
import numpy as np
l = [[]]
for i, sent in enumerate(dev_sentence):
    for j, word in enumerate(sent):
        l[i].append(np.argmax(res[i][j]))
    if i<len(dev_sentence)-1:
        l.append([])
        
dev_labels = [[]]
for i, sent in enumerate(dev_sentence):
    for j, word in enumerate(sent):
        dev_labels[i].append(idx2tag[l[i][j]])
    if i<len(dev_sentence)-1:
        dev_labels.append([])

with open('dev_pred_f.txt', 'w') as f:
    for line in dev_labels:
        for word in line:
            f.write(f"{word}\n")
        f.write('\n')

In [48]:
punctuations = '''“”!()-[]{};:'"\,<>./?@#$%^&*_~�।’‘1234567890''' #we can add suitable extra punctuation all the time
def remove_punctuation(d):
    review = d.replace('\n', '')
    no_punct = ""
    for char in review:
        if char not in punctuations:
            no_punct = no_punct + char
    return no_punct

In [98]:
def input_file_test(fileName):
    word = [[]]
    sentences = [[]]
    target = [[]]
    count_class = {}
    i=0
    with open(fileName, 'r') as file:
        for line in file:
            l = line.split()
            if len(l):
                if l[0] not in punctuations:
                    l[0] = remove_punctuation(l[0])
                    if l[0] not in unique_words:
                        word[i].append('unknown')
                    else:
                        word[i].append(l[0])
                else:
                    if l[0] not in unique_words:
                        word[i].append('unknown')
                    else:
                        word[i].append(l[0])
            else:
                word.append([])
                i+=1
    
    return word

In [99]:
test_word = input_file_test('test.txt')

In [100]:
x_test = [[model_word_to_vec.wv.get_vector(x) for x in s] for s in test_word]
x_test = pad_sequences(maxlen = mxlen+5, sequences=x_test, padding='post', value=len_unique_words-1)

In [101]:
x_input_test = [elem for twod in x_test for elem in twod]

In [102]:
y_test_pred = rf.predict(x_input_test)

In [106]:
# initializing K 
K = 55
  
# Convert 2D list to 3D at K slicing
# Using loop
res = []
subl = []
cnt = 0
for sub in y_test_pred:
    subl.append(sub)
    cnt = cnt + 1
    if cnt >= K:
        res.append(subl)
        subl = []
        cnt = 0

In [107]:
len(res)

13215

In [108]:
l = [[]]
for i, sent in enumerate(test_word):
    for j, word in enumerate(sent):
        l[i].append(np.argmax(res[i][j]))
    if i<len(test_word)-1:
        l.append([])

dev_labels = [[]]
for i, sent in enumerate(test_word):
    for j, word in enumerate(sent):
        dev_labels[i].append(idx2tag[l[i][j]])
    if i<len(test_word)-1:
        dev_labels.append([])
        
with open('test_pred_labels_f.txt', 'w') as f:
    for line in dev_labels:
        for word in line:
            f.write(f"{word}\n")
        f.write('\n\n')